# Modelos de Series de Tiempo

Presentado por Daniel Ortíz

**Recapitulando:**

El *nivel* de una serie de tiempo es el promedio de los datos sin importar el orden de los mismos, brinda una estimación de los valores con los que se trabaja.

La *tendencia* es el comportamiento general de la Serie de Tiempo.

El *ruido* son valores que se desvían de ese comportamiento general.

Grandes perturbaciones en los datos dificultan las predicciones.

Algunas técnicas para evaluar la tendencia de una serie de tiempo con la regresión lineal:

- Sólo regresión
- Regresión y quitando anomalías (outliers)
- Combinada con el promedio móvil

### Precio con nivel
![Precio con promedio](./graficas/Precio_promedio.jpg)

### Precio sin nivel
![Precio sin promedio](./graficas/Precio_sin_promedio.jpg)

### Precio con tendencia
![Precio con tendencia](./graficas/Precio_tendencia.jpg)

### Precio sin tendencia
![Precio sin tendencia](./graficas/Precio_sin_tendencia.jpg)

La línea recta de la regresión lineal no logra capturar la nueva tendencia después de eventos pronunciados

![Precio con predicción](./graficas/Precio_RL.jpg)

La **regresión lineal** se puede aplicar a todos los datos, pero los **modelos autorregresivos** son exclusivos de series de tiempo, pues aprovechan el orden temporal de los datos para funcionar.

Si un valor de una variable en el futuro puede depender de su valor en el pasado, se puede utilizar un modelo autorregresivo.

## Modelo Autoregresivo (AR)

En el caso de tener una variable dependiente $y$, el modelo autoregresivo se puede expresar como:

\begin{align}
y(t_n) = a_0 + a_1 y(t_{n-1}) + a_2y(t_{n-2}) + .... a_m y(t_{n-m})
\end{align}

En donde las $a$'s son constantes a definir que corresponden a los pesos del modelo

Las $y$'s del lado derecho de la ecuación corresponden a valores previos de la serie 

Y $m$ es el orden del modelo autoregresivo.

Esto se reduce a una regresión lineal múltiple, en donde las variables corresponden a los valores previos de la variable a predecir; por lo tanto, se tiene un modelo que predice el futuro utilizando solamente valores del pasado.

Un modelo autocorrelacionado es un modelo de una variable contra sí misma pero con valores del pasado.

## Modelo Moving Average (MA)

El modelo MA es un modelo que realiza una regresión lineal sobre los términos residuales anteriores y se puede escribir de la siguiente forma.

\begin{align}
y(t) = \mu + \theta_0 \epsilon_t + \theta_1 \epsilon_{t-1} + ... \theta_m \epsilon_{t-m}
\end{align}

En donde $y$ es el valor a predecir, $\theta_n$ son las constantes a determinar y $\epsilon_n$ es el término del error que se utiliza para ajustar el modelo.

Este modelo *no* es el rolling mean o el SMA.

Este modelo es más complejo que el autorregresivo, debido a que se utilizan los residuos del pasado en vez de los valores del pasado para la predicción, los residuos no están presentes de forma directa en los datos, por lo cual su ajuste requiere de la implementación de métodos no lineales, ya que se necesitan los residuos para calcular el modelo y se necesita el modelo para encontrar los residuos.

In [ ]:
#Importación de paquetes
import warnings
import pickle
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import ParameterGrid
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
pd.plotting.register_matplotlib_converters()
from scipy.stats import boxcox
import yfinance as yf
yf.pdr_override()
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr

warnings.filterwarnings("ignore")

def mean_square_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
#Activo a analizar
assets = ["AMZN"]
start_date = "2019-01-01"
end_date = "2021-01-01"

In [ ]:
#Creación de DataFrame
df=pd.DataFrame()
for asset in assets:
    df_asset = pdr.get_data_yahoo(asset, start=start_date, end=end_date)["Adj Close"]#El activo se conecta a Yahoo y hace lectura
    df_asset = df_asset.to_frame(name=asset) #Descarga la información
    df = pd.concat([df, df_asset], axis=1, sort=False) #al data frame se le agrega el nuevo activo

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace = True)
df.rename(columns = {'index': 'date', 'Date':'date', 'AMZN':'close'}, inplace = True)
df.head()

In [ ]:
#Visualización
plt.figure(figsize=(10,5)) 
plt.plot(df['date'], df['close'])
plt.title("Precio de cierre de AMZN. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.savefig("img/Precio AMZN.jpg")
plt.show()

Con el modelo **AR** se tomará el valor del día anterior para predecir el valor del día siguiente.

¿Cómo se hace?

In [ ]:
new = []
for i in df.index:
    if i < 1:
        new.append(np.nan)
    else:
        new.append(df.loc[i - 1, 'close'])

In [ ]:
df['close_1'] = new

In [ ]:
df.head()
# El valor del día anterior en columna nueva

In [ ]:
df.drop(columns = "close_1", inplace = True)

In [ ]:
shifted_df = pd.concat([df, 
                        df['close'].shift(), 
                        df['close'].shift(2), 
                        df['close'].shift(3)],axis=1)

shifted_df.columns = ['date', 'close', 'close_1', 'close_2', 'close_3']

In [ ]:
shifted_df.head()
# Con shift se toman valores anteriores

In [ ]:
shifted_df.dropna(axis = 0, inplace = True)

Aplicando un modelo de Regresión Lineal con las variables nuevas se puede predecir la variable en cuestión. (Predecir 'close' con 'close_1' y 'close_2')

In [ ]:
test_sample = 50 #Muestra de prueba
train_df = shifted_df.iloc[:(shifted_df.shape[0] - test_sample),:] #Set de entrenamiento
test_df = shifted_df.iloc[(shifted_df.shape[0] - test_sample):,:] #Set de prueba

In [ ]:
print(train_df.shape, test_df.shape, shifted_df.shape) #Dimensiones de sets

In [ ]:
#Primero tenemos que convertir los datos a una x equivalente.
dummy = np.linspace(0,shifted_df.shape[0] - 1, shifted_df.shape[0]).reshape(-1,1)
dummy2 = np.linspace(0,train_df.shape[0] - 1, train_df.shape[0]).reshape(-1,1)

#Entrenamiento del modelo
reg = LinearRegression().fit(train_df[['close_1', 'close_2']], train_df['close']) #Predecir la Y con las X's 

In [ ]:
dummy3 = np.linspace(train_df.shape[0] - 1, df.shape[0] - 1, test_df.shape[0]).reshape(-1,1)
test_results = reg.predict(test_df[['close_1', 'close_2']])

In [ ]:
results2 = reg.predict(shifted_df[['close_1', 'close_2']])
#plt.plot(train_df['date'],train_df['close'])
plt.figure(figsize=(10,5)) 
plt.plot(shifted_df['date'], shifted_df['close'], label = 'Datos')
plt.plot(shifted_df['date'], results2, label = 'Modelo')
plt.title('Predicciones modelo AR(2)')
plt.ylabel('Precio de la Acción (USD)')
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Predicciones AR.jpg")
plt.show()

In [ ]:
#Cálculo de score
reg.score(test_df[['close_1', 'close_2']], test_df['close'])

In [ ]:
results2 = reg.predict(shifted_df[['close_1', 'close_2']])
#plt.plot(train_df['date'],train_df['open'])
plt.plot(test_df['date'], test_df['close'], label = 'Set de Prueba')
plt.plot(test_df['date'], test_results, label = 'Predicciones')
plt.title('Predicciones modelo AR(2)')
plt.ylabel('Precio de la Acción (USD)')
plt.xlabel("Fecha")
plt.legend()
plt.show()

El modelo se ajusta casi perfecto al set... ¿Por qué?

Puede parecer que este modelo es prácticamente perfecto, ya que puede predecir de forma casi impecable el set de entrenamiento, al evaluar los puntos en la regresión, se observa que hay una coincidencia muy grande entre los datos, por lo que se podría concluir que este modelo es significativamente superior al modelo de regresión lineal simple.

### Sesgo "Forward Bias"

Esto no refleja el verdadero resultado del modelo y no representa una verdadera predicción de valores futuros, debido a un sesgo llamado "sesgo del futuro" en el cuál, se utilizan datos que no están disponibles en su momento para realizar predicciones (El modelo es bueno porque se utilizan 'Datos del futuro').


In [ ]:
#Últimos valores del set de entrenamiento
train_df.tail()

Se entrena el modelo hasta la semana del 20 de octubre del 2020; por lo tanto, para el modelo y la perspectiva de entrenamiento, se puede decir esta es la fecha del dia de hoy, por lo que los valores del set de prueba se desconocen.

En este caso no se podría realizar una predicción a los datos del set de prueba por que no existen, no se tienen en el momento, por lo que no se puede realizar esa evaluación.

**¿Es posible predecir valores a futuro sin conocer el set de prueba?**

In [ ]:
#Primeros valores de set de prueba
test_df.head()

Si: El último valor del set de entrenamiento se convierte en el primer retraso y el penúltimo valor se convierte en el segundo retraso, con estos datos es posible predecir un valor a futuro.

Una vez que tenemos el primer valor a futuro, este se convierte en el primer retraso y el último del set de entrenamiento se convierte en el segundo retraso.

\begin{align}
y(t+1) = a_0 + a_1 y(t) + a_2y(t-1) \\
y(t+2) = a_0 + a_1 y(t+1) + a_2 y(t)
\end{align}

A este proceso se le llama "one-step forward forecasting", ya que va realizando el ajuste un paso a la vez y esto permite una generación de lo que podría ser el valor futuro de nuestros datos, cabe recalcar que debido a la naturaleza estocastica de las variables y la dependencia con respecto a pocos términos atrasados, no se recomienda realizar predicciones para periodos muy largos de tiempo.

| date | close | close_1 | close_2 |
| --------- | --------- | --------- | --------- |
| 2020 - 03 - 18 | 147 | 138 | 125 |
| 2020 - 03 - 19 | 148 | 147 | 138 |
| 2020 - 03 - 20 | 150 | 148 | 147 |

In [ ]:
lag_1 = test_df['close_1'][test_df.index[0]]
lag_2 = test_df['close_2'][test_df.index[0]]
forecast = []
for i in test_df.index:
    val = reg.predict(np.array([lag_1, lag_2]).reshape(1,-1))
    forecast.append(val)
    lag_2 = lag_1
    lag_1 = val

In [ ]:
reg.score(test_df[['close_1', 'close_2']], forecast )

In [ ]:
plt.plot(dummy3, test_df['close'], label = "Datos de Prueba" )
plt.plot(dummy3, forecast, label = 'Forecast')
plt.xlabel("Observación")
plt.ylabel("Precio de Acción (USD)")
plt.legend()
plt.title('Forecast modelo AR(2)')
plt.show()

In [ ]:
plt.plot(dummy3, test_df['close'], label = "Datos de Prueba" )
plt.plot(dummy3, forecast, label = 'Forecast')
plt.xlabel("Observación")
plt.ylabel("Precio de Acción (USD)")
plt.legend()
plt.title('Forecast modelo AR(2)')
plt.show()

In [ ]:
plt.plot(dummy3, test_df['close'], label = "Datos de Prueba" )
plt.plot(dummy3, forecast, label = 'Forecast')
plt.xlabel("Observación")
plt.ylabel("Precio de Acción (USD)")
plt.legend()
plt.title('Forecast modelo AR(2)')
plt.show()

In [ ]:
plt.plot(dummy3, test_df['close'], label = "Datos de Prueba" )
plt.plot(dummy3, forecast, label = 'Forecast')
plt.xlabel("Observación")
plt.ylabel("Precio de Acción (USD)")
plt.legend()
plt.title('Forecast modelo AR(2)')
plt.show()

In [ ]:
plt.plot(dummy3, test_df['close'], label = "Datos de Prueba" )
plt.plot(dummy3, forecast, label = 'Forecast')
plt.xlabel("Observación")
plt.ylabel("Precio de Acción (USD)")
plt.legend()
plt.title('Forecast modelo AR(2)')
plt.show()

### ¿Que podemos concluir de este modelo?

- 1. La predicción final es muy sensible al punto de corte del set de entrenamiento.
- 2. La predicción final suele ser precisa a corto plazo.
- 3. Al ir actualizando el modelo, la predicción se adapta a los cambios, lo cual lo vuelve muy dinámico.

Este modelo es una mejora considerable comparada con los modelos previamente utilizados; por lo tanto, podemos concluir que la decisión de aplicar un modelo autoregresivo en nuestros datos es la correcta ya que aproxima el comportamiento del set de prueba.

¿Este es el mejor modelo que podemos crear? Quizá no.

- 1. Se seleccionó de forma muy arbitraria el uso de 2 valores previos para la predicción del valor futuro.
- 2. Hay otro tipo de modelos que se pueden utilizar en la predicción de los datos futuros.
- 3. Hay una metodología establecida para encontrar el mejor modelo para una serie temporal.